### IMPORT

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

import warnings
warnings.filterwarnings("ignore")


In [2]:
dataset_filename = 'Online Retail.xlsx'

In [4]:
dataset = pd.read_excel(dataset_filename, sheet_name='Online Retail')

### DATA INFO

In [5]:
dataset.shape

(541909, 8)

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [20]:
dataset.keys()

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [22]:
dataset.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [ ]:
# check for missing values
dataset.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

### PREPROCESSING

### TRAIN/TEST SET

In [41]:
def describe(df, pred=None):
    """this function describes a dataframe basic information"""
    obs = df.shape[0]
    types = df.dtypes
    counts = df.apply(lambda x: x.count())
    uniques = df.apply(lambda x: [x.unique().tolist()])
    nulls = df.apply(lambda x: x.isnull().sum())
    distincts = df.apply(lambda x: x.unique().shape[0])
    missing_ratio = (df.isnull().sum()/ obs) * 100
    
    # apply skew and kurtosis to numeric column
    numeric_cols = df.select_dtypes(include=['number']).columns
    skewness = df[numeric_cols].skew()
    kurtosis = df[numeric_cols].kurt()

    # apply skew and kurtosis to datetime column - fill with NaN
    skewness = skewness.reindex(df.columns, fill_value=np.nan)
    kurtosis = kurtosis.reindex(df.columns, fill_value=np.nan)


    print('Data shape:', df.shape)
    
    # if pred is None:
    #     cols = ['types', 'counts', 'distincts', 'nulls', 'missing ratio', 'uniques', 'skewness', 'kurtosis']
    #     output = pd.concat([types, counts, distincts, nulls, missing_ratio, uniques, skewness, kurtosis], axis = 1, sort=True)
    # else:
    #     corr = df.corr()[pred]
    #     output = pd.concat([types, counts, distincts, nulls, missing_ratio, uniques, skewness, kurtosis, corr], axis = 1, sort=True)
    #     corr_col = 'corr '  + pred
    #     cols = ['types', 'counts', 'distincts', 'nulls', 'missing ratio', 'uniques', 'skewness', 'kurtosis', corr_col ]
    
    # output.columns = cols
    # dtypes = output.types.value_counts()

    # Combine all into a DataFrame and transpose so each row is a column in the original df
    output = pd.DataFrame({
        'types': types,
        'counts': counts,
        'distincts': distincts,
        'nulls': nulls,
        'missing ratio': missing_ratio,
        'uniques': uniques,
        'skewness': skewness,
        'kurtosis': kurtosis
    })

    # If a prediction column is provided, compute correlation
    if pred is not None:
        # Only use numeric columns for correlation
        corr = df.corr(numeric_only=True)[pred].reindex(df.columns, fill_value=np.nan)
        output['corr ' + pred] = corr

    print('___________________________\nData types:\n\n',output.types.value_counts())
    print('___________________________')

    return output


details = describe(dataset)
display(details.sort_values(by='missing ratio', ascending=False))


Data shape: (541909, 8)


ValueError: 2

In [43]:
import pandas as pd
import numpy as np

def describe(df, pred=None):
    """Safe DataFrame summary function with handling for non-numeric and datetime types."""

    obs = df.shape[0]

    # Basic stats
    types = df.dtypes
    counts = df.count()
    nulls = df.isnull().sum()
    distincts = df.nunique(dropna=False)
    missing_ratio = (nulls / obs) * 100

    # Optional: count of unique values (not the actual list)
    unique_count = df.apply(lambda x: len(x.unique()) if x.notnull().any() else 0)

    # Skewness/Kurtosis only for numeric columns
    numeric_cols = df.select_dtypes(include=['number']).columns
    skewness = df[numeric_cols].skew().reindex(df.columns, fill_value=np.nan)
    kurtosis = df[numeric_cols].kurt().reindex(df.columns, fill_value=np.nan)

    # Correlation if target is given
    if pred is not None and pred in df.columns:
        corr = df.corr(numeric_only=True)[pred].reindex(df.columns, fill_value=np.nan)
        corr_name = 'corr ' + pred
    else:
        corr = pd.Series(np.nan, index=df.columns)
        corr_name = 'correlation'

    # Build final output DataFrame
    output = pd.DataFrame({
        'types': types,
        'counts': counts,
        'distincts': distincts,
        'nulls': nulls,
        'missing ratio': missing_ratio,
        'unique count': unique_count,
        'skewness': skewness,
        'kurtosis': kurtosis,
        corr_name: corr
    })

    # Summary print
    print('Data shape:', df.shape)
    print('___________________________\nData types:\n\n', output['types'].value_counts())
    print('___________________________')

    return output


details = describe(dataset)
display(details.sort_values(by='missing ratio', ascending=False))


Data shape: (541909, 8)
___________________________
Data types:

 types
object            4
float64           2
int64             1
datetime64[ns]    1
Name: count, dtype: int64
___________________________


,types,counts,distincts,nulls,missing ratio,unique count,skewness,kurtosis,correlation
CustomerID,float64,406829,4373,135080,24.926694,4373,0.029835,-1.179982,NaN
Description,object,540455,4224,1454,0.268311,4224,NaN,NaN,NaN
InvoiceNo,object,541909,25900,0,0.000000,25900,NaN,NaN,NaN
StockCode,object,541909,4070,0,0.000000,4070,NaN,NaN,NaN
Quantity,int64,541909,722,0,0.000000,722,-0.264076,119769.160031,NaN
InvoiceDate,datetime64[ns],541909,23260,0,0.000000,23260,NaN,NaN,NaN
UnitPrice,float64,541909,1630,0,0.000000,1630,186.506972,59005.719097,NaN
Country,object,541909,38,0,0.000000,38,NaN,NaN,NaN


### RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

